# Detroit Blight
A study notebook

Overall plan <br>
1) Load and explore incident data from the various files: violations, 311 calls, crime <br>
2) Define 'houses' as  geographic entities, for example a 10 x 10 meter squares. Allocate incidents to houses. <br>
3) Allocate known demolition permits to houses. Houses marked for demolition are regarded as blighted. <br>
4) Train a model to predict which houses will be marked for demolition (=are blighted) <br>

## Data
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-blight-violations.csv
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-311.csv
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-crime.csv
https://github.com/uwescience/datasci_course_materials/blob/master/capstone/blight/data/detroit-demolition-permits.tsv





In [ ]:
library(data.table)

# Program idea: fit incident data to data frame. Select only relevant columns, and always use same names for them. 
# From incident data, derive unique locations of a given area = houses. House data: HouseId, Lat, Lon.

loadDf <- function(fileName) {
    basedir <-
        "C:/Users/setup/Documents/coursera/uw/datascience/capstone"
    datadir <- paste0(basedir, "/data")
    codedir <- paste0(basedir, "/code")
    fullName <- paste0(datadir, "/", fileName)
    df <- read.csv(fullName, stringsAsFactors = FALSE)
    return(df)
}

getBViol <- function(){
    bv <- loadDf("detroit-blight-violations.csv")
    return(bv)
}

formatBv <- function(bv) {
    bv <- subset(
        bv,select = c(
            "ViolationCode",
            "ViolationAddress","ViolationStreetNumber", "ViolationStreetName"
        )
    )
    extractCoord <- function(bvsRow) {
        # bvsRow is like ..., 2566 GRAND BLVD\nDetroit, MI\n(42.36318237000006, -83.09167672099994)
        rowStr <- unlist(strsplit(toString(bvsRow),split = "\n"))[3]
        # Now we have something like (42.32544917300004, -83.06413908999997)
        coord <- gsub("[()]","",rowStr)
        coord <- unlist(strsplit(coord,split = ","))
        return(coord)
    }
    coord <- t(apply(bv,1,extractCoord))
    # Add latitude and longitude as first columns
    bv <- cbind(as.numeric(coord[,1]),as.numeric(coord[,2]),bv)
    names(bv) <-
        c("Lat","Lon","IncType","VAddress","VStrNr","VStrName")
    bv <- bv[,1:6]
} # formatBv

formatBvAddress <- function(bv){
    # Check column names before calling this function!
    bv$ViolationAddress <- gsub("\n",",",bv$ViolationAddress)
    # bv$MailingAddress <- gsub("\n",",",bv$MailingAddress)
    return(bv)
}

getD311 <- function() {
    d311 <- loadDf("detroit-311.csv")
    # We'll ignore some alerts, as they are not specific to a house
    # nor caused by the house owner
    ignore <-
        c(
            "Water Main Break", "Fire Hydrant Issue", "Traffic Signal Issue",
            "Potholes", "Test (internal use only, public issue)",
            "Customer Service (internal use only, private issue)",
            "Graffiti Abatement (internal use only, public issue)"
        )
    # d311 <- d311[!(d311$issue_type %in% ignore),]
    u311type <- unique(d311$issue_type,use.names = FALSE)
    d311 <- subset(
        d311,!(d311$issue_type %in% ignore),
        select = c(
            issue_type,ticket_closed_date_time,ticket_created_date_time,
            address,lat,lng
        )
    )
    # assign("u311type",u311type,envir = .GlobalEnv)
    n311 <- names(d311)
    n311[1] <- "IncType"
    n311[5] <- "Lat"
    n311[6] <- "Lon"
    names(d311) <- n311
    d311$Lat <- as.numeric(d311$Lat)
    d311$Lon <- as.numeric(d311$Lon)
    return(d311)
}

getCrime <- function(){
    cr <- loadDf("detroit-crime.csv")
    cr <- subset(cr,select=c("CATEGORY", "LAT", "LON"))
    names(cr) <- c("IncType", "Lat", "Lon")
    return(cr)
}

buildHousesFromDf <- function(df, houses, precision){
    # We'll aggregate all incidents from any df to a set of houses (coordinates) irrespective of incident type.
    # House structure: (HouseId, Lat, Lon). Lat and Lon represent the center of the house, which is about 11m x 11m.
    hCoord <- data.frame(Lat=houses$Lat, Lon=houses$Lon)
    dCoord <- data.frame(Lat=round(df$Lat,digits = precision), Lon=round(df$Lon,digits = precision))
    # Remove NA values
    naVector <- is.na(dCoord$Lat+dCoord$Lon)
    dCoord <- dCoord[!naVector,]
    coord <- rbind(hCoord,dCoord)
    rm(df,hCoord,dCoord)
    # Remove added duplicates
    coord <- coord[!duplicated(coord),]
    houses <- coord
    rm(coord)
    return(houses)
}

## 2) Build houses
Now that the functions are defined, let's actually build the houses.

In [ ]:
buildHouses <- function(){
    # Load each file and allocate incident coordinates to houses
    bv <- getBViol()
    bv <- formatBv(bv)
    houses <- NULL
    houses <- buildHousesFromDf(bv, houses, 4)
    rm(bv)
    d311 <- getD311()
    houses <- buildHousesFromDf(d311, houses, 4)
    rm(d311)
    cr <- getCrime()
    houses <- buildHousesFromDf(cr, houses, 4)
    houses <- houses[order(houses$Lat, houses$Lon),]
    row.names(houses) <- 1:nrow(houses)
    houses <- cbind(row.names(houses), houses)
    names(houses) <- c("HouseId", "Lat", "Lon")
    houses$HouseId <- as.character(houses$HouseId)
    houses <- data.table(houses)
    setkey(houses,Lat,Lon)
    return(houses)
}

    houses <- buildHouses()
    write.csv(file = "houses.csv",x = houses,row.names = TRUE)
    assign("houses",houses,.GlobalEnv)

# 3) Get demolition permits
Load demolition permit data. Assign permits to previously defined houses. As a result, we have a list of demolition permits with a HouseId, if available. Many of the permits don't belong to any of the houses we've constructed above.
Again, first define functions, then finally use them.


In [ ]:
getDemolition <- function(){
        # if(is.null(dem)){
            dem <- read.csv(file=
                                "C:/Users/setup/Documents/coursera/uw/datascience/capstone/data/detroit-demolition-permits.tsv",
                            sep = "\t", stringsAsFactors = FALSE)
        # }
#         dem <- subset(dem, select=c(PERMIT_ISSUED, CASE_TYPE, BLD_PERMIT_TYPE, site_location))
        return(dem)
}

formatDemolition <- function(dem){
        extractCoordAdd <- function(row){
            # site_location seems to contain the address and geocoordinates of the building to be demolished
            loc <- row["site_location"]
            # loc is like "4331 BARHAM\nDetroit, MI\n(42.394106, -82.9474)"
            loc <- unlist(strsplit(toString(loc),split = "\n"))
            str <- loc[1]
            cty <- loc[2]
            crd <- loc[3]
            # Now we have something like (42.32544917300004, -83.06413908999997)
            crd <- gsub("[()]","",crd)
            crd <- unlist(strsplit(crd,split = ","))
            lat <- crd[1]
            lon <- crd[2]
            coord <- c(lat, lon, str, cty)
            return(coord)
        }
        dnam <- names(dem)
        coord <- t(apply(dem, 1, extractCoordAdd))
        dem <- cbind(as.numeric(coord[,1]), as.numeric(coord[,2]), coord[,3], coord[,4], dem)
        dnam <- c("Lat", "Lon", "Street", "City", dnam)
        names(dem) <- dnam
        dem$Street <- as.character(dem$Street)
        dem$City <- as.character(dem$City)
        return(dem)
}

prepareDemolition <- function(){
    dem <- getDemolition()
    dem <- formatDemolition(dem)
    dem <- data.table(dem)
    # Some 900 entries have NA for coordinates. We need to treat them separately with geocoding.
    demToGeocode <- dem[is.na(dem$Lat),]
    # demToGeocode <- head(demToGeocode,10)
    # Remove from dem the entries that need to be geocoded 
    dem <- dem[!(is.na(dem$Lat)),]
    
    addresses <- paste0(demToGeocode$Street, " ",gsub(",", "",demToGeocode$City))
    geocoded <- data.frame()
    # Start the geocoding process - address by address. geocode() function takes care of query speed limit.
    for (ii in (1:length(addresses))){
        if (ii %% 50 == 0){
            print(paste("Working on index", ii, "of", length(addresses)))    
        }
        #query the google geocoder - this will pause here if we are over the limit.
        result = geoCodeAddress(addresses[ii]) 
        # print(result$status)     
        result$index <- ii
        #append the answer to the results file.
        geocoded <- rbind(geocoded, result)
        #save temporary results as we are going along
        # saveRDS(geocoded, tempfilename)
    }
    # demToGeocode$accuracy <- geocoded$accuracy
    demToGeocode$Lat <- geocoded$Lat
    demToGeocode$Lon <- geocoded$Lon
    # Remove the most obvious geographic outliers
    demToGeocode <- demToGeocode[Lat %between% c(42,43) & Lon %between% c(-83.5,-81)]
    # Finally add the geocoded entries to dem
    dem <- rbind(dem,demToGeocode)
    return(dem)
}

geoCodeAddress <- function(address){
    geo_reply = geocode(address, output='all', messaging=TRUE, override_limit=TRUE)
    #now extract the bits that we need from the returned list
    answer <- data.frame(Lat=NA, Lon=NA, accuracy=NA, formatted_address=NA, address_type=NA, status=NA)
    answer$status <- geo_reply$status
    #if we are over the query limit - want to pause for an hour
    while(geo_reply$status == "OVER_QUERY_LIMIT"){
        print("OVER QUERY LIMIT - Pausing for 1 hour at:") 
        time <- Sys.time()
        print(as.character(time))
        Sys.sleep(60*60)
        geo_reply = geocode(address, output='all', messaging=TRUE, override_limit=TRUE)
        answer$status <- geo_reply$status
    }
    if (geo_reply$status != "OK"){
        return(answer)
    }   
    #else, extract what we need from the Google server reply into a dataframe:
    answer$Lat <- geo_reply$results[[1]]$geometry$location$lat
    answer$Lon <- geo_reply$results[[1]]$geometry$location$lng   
    if (length(geo_reply$results[[1]]$types) > 0){
        answer$accuracy <- geo_reply$results[[1]]$types[[1]]
    }
    answer$address_type <- paste(geo_reply$results[[1]]$types, collapse=',')
    answer$formatted_address <- geo_reply$results[[1]]$formatted_address
    return(answer)
}


In [ ]:
# Time to put it all together!
# Get and format the demolition permits
dem <- prepareDemolition()

demolitionHouses <- function(dem, houses){
    h <- data.table(houses)
    setkey(h,Lat,Lon)
    # Link demolition permits to houses
    d <- apply(dem, 1, findInHouses3,h)
    dem <- cbind(d, dem)
    names(dem)[names(dem)=="d"] <- "house"
    dem
}

findInHouses3 <- function(row, houses) {
    # houses must be a data.table. Returns HouseId (an integer) or NA
    rlat <- round(as.numeric(row["Lat"]),digits=4)
    rlon <- round(as.numeric(row["Lon"]),digits=4)
    res <- houses[list(rlat,rlon)]
    res$HouseId
}

# Assign them to houses
dh <- demolitionHouses(dem,houses)

### Shortcut
Using the file I've saved from my dataframes, load dh

In [ ]:
dh <- read.csv("houses_with_demolition.csv",stringsAsFactors = FALSE)
# Checked in Carto that only 1 of these may fall a few meters outside Detroit --> ignore error
dh <- data.table(dh)

## 4) Prepare demolition permits for training and testing
Many demolition permits are not associated to any of our houses. In other words there are no incidents linked to them. Let's disregard them.

In [ ]:
> gh <- dh[!is.na(house)]
> sgh <- subset(gh,select=c("house","Lat","Lon","PERMIT_ISSUED","LEGAL_USE","PARCEL_SIZE","STORIES"))
rm(dh,gh)
> nrow(sgh)

There should be 3573 demolition permits assigned to houses (sgh).
Check for duplicates ie. multiple permits to the same house:

In [ ]:
> sum(duplicated(sgh$house))
[1] 1090
> sum(duplicated(sgh$house, sgh$Lat, sgh$Lon))
[1] 1090

Finally remove duplicates and prepare the train and test sets.

In [ ]:
sgh <- sgh[!duplicated(sgh$house),] 
n <- nrow(sgh)
train <- sample(1:n, size=round(0.7*n),replace=FALSE)
demtrain <- sgh[train,]
demtest <- sgh[-train,]
# Some crude validity tests
sum(duplicated(sgh$house))
max(sgh$Lat)
max(sgh$Lon)

bHouses <- data.table(HouseId = sgh$house, ToDemolition = TRUE)
nbHouses <- houses[!(HouseId %in% sgh$house)]
nbHouses <- data.table(HouseId = as.integer(nbHouses$HouseId), ToDemolition = FALSE)
nbHouses <- nbHouses[sample(1:nrow(nbHouses), size=nrow(bHouses), replace=FALSE),]
# as result, we have nrow(bHouses) = nrow(nbHouses) = 2483

ttHouses <- rbind(bHouses, nbHouses)
ttHouses[order(ttHouses$HouseId),]
# Ordering by HouseId should distribute the blighted/nonblighted houses fairly randomly, 
# as the house numbers were generated with no reference to blight


# 5) Number of incidents per house in training/test set

In [ ]:
bvc <- loadDf("bvCounts.csv") #A result of previous aggregation of incidents per Lat,Lon

Alternatively, here's how the object in bvCounts was built:

In [ ]:
bv <- loadDf("Detroitbviolfromcartocsv.csv")
bv <- subset(bv, select=c("lat", "lon"))
bv$lat <- round(bv$lat,digits=4)
bv$lon <- round(bv$lon,digits=4)
bv <- data.table(bv)
bv <- bv[, ":=" (Count = .N), by= list(lat,lon)]
# The data table operator .N adds the number of incidents in the by group to every member of the group.
# It did not remove duplicates (aggregate to one member per group)
bv <- unique(bv)
names(bv)=c("Lat", "Lon", "Count")
setkey(bv,Lat,Lon)

train <- loadDf("trainSet.csv")
train <- data.table(train)
setkey(train,Lat,Lon)
# Join the data tables
bt <- bv[train]
names(bt)
# [1] "Lat"          "Lon"          "Count"        "HouseId"      "ToDemolition"


### 6) Build simple model
Build the model based on solely the number of incidents per case in the training set.

In [ ]:
btree <- tree(ToDemolition ~ Count, data= bt)
cv.tree(btree, , prune.tree,K=5)
plot(tree)
text(tree)

The resulting model is very simple: if the number of incidents >0.5 (in practice if there are incidents) then predict TRUE = to be demolished = blighted.
The precision is not too good.